Есть датфрейм с authors, text, annotation и title. Текст предобработан, стоп-слова удалены. У одной статьи может быть несколько авторов, тогда они записаны в одной ячейке authors и разделены запятой. 

Функции:
- для вычисления всех соавторов одного человека по ФИО, они могут быть записаны в разных ячейках датафрейма. 

- функция для векторизации текста.  

- функция для вычисления коэффициента схожести.

- функция для рекомендации новых соавторов. 

В функцию для рекомендации новых соавторов подаётся на вход ФИО автора. На выходе ФИО автора, названия всех статей в которых он участвовал, список его соавторов, список трёх рекомендуемых новых авторов/коллективов авторов, названия их статей и коэффициент близости. 

Ограничения:

- Нельзя рекомендовать автора самому себе 
- Нельзя рекомендовать автору его текущих соавторов. 

Особенности:

Необходимо предусмотреть ситуацию когда рекомендуется несколько человек, написавших одну и ту же статью, при совпадении коэффициента близости у новых соавторов необходимо объединять их ФИО через запятую, писать их общую статью один раз и на 2 и 3 месте писать новых рекомендуемых авторов. При совпадении 2 и 3 рекомендации также следует их объединять и на третье место выводить следующих по коэффициенту схожести коллектив авторов/автора.

In [26]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pprint

In [30]:
df = pd.read_excel('date_it_lemm.xlsx')

In [5]:
#df['authors'] = df['authors'].str.split(',') #авторов через запятую разделяем

In [32]:
df

,date,title,link,annotation,authors,text,UDK,ESCI,GeoRef,ВАК,...,RSCI,WOS,AGRIS,PubMed,MathSciNet,Scopus,zbMATH,CAS,count_words,topic
0,2019,Проблема обработки данных георадиолокационных ...,/article/n/problema-obrabotki-dannyh-georadiol...,в статье рассматривается проблема обработки да...,Поротова Варвара Петровна,проблема обработка данные георадиолокационный ...,NaN,0,0,0,...,0,0,0,0,0,0,0,0,776,IT
1,2019,Understanding and designing modern high-reliab...,/article/n/understanding-and-designing-modern-...,the purpose of this paper is to describe as Sp...,"Zemtsov Andrey Nikolaevich, Tran Dung Khanh",проектирование современный высоконадёжный сеть...,NaN,0,0,0,...,0,0,0,0,0,0,0,0,1421,IT
2,2019,разработка аутентификационной модели для обл...,/article/n/razrabotka-autentifikatsionnoi-mode...,рассмотрены методы и модели безопасного исполь...,"Вишняков Александр Сергеевич, Макаров Анатолии...",разработка аутентификационный модель облачный ...,УДК 331.225.3,0,0,0,...,0,0,0,0,0,0,0,0,2719,IT
3,2019,Разработка модели информационных потоков в сис...,/article/n/razrabotka-modeli-informatsionnyh-p...,в данной работе представлена модель информаци...,Жуков Алексей Викторович,разработка модель информационный поток система...,NaN,0,0,0,...,0,0,0,0,0,0,0,0,2256,IT
4,2019,Автоматизированная система управления солнечны...,/article/n/avtomatizirovannaya-sistema-upravle...,в данной статье рассмотрена возможность приме...,Зеленцов Денис Сергеевич,автоматизированный система управление солнечны...,NaN,0,0,0,...,0,0,0,0,0,0,0,0,1132,IT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2305,2024,Программный комплекс распределенного тестирова...,/article/n/programmnyy-kompleks-raspredelennog...,"Статья посвящена тестированию веб-приложений, ...","П А. Редкин, А С. Алёшкин",программный комплекс распределённый тестирован...,NaN,0,0,1,...,0,0,0,0,0,0,0,0,3839,IT
2306,2024,Генерация файлов формата Moodle XML на основе ...,/article/n/generatsiya-faylov-formata-moodle-x...,Цель предлагаемого исследования разработать ал...,Горожанов Алексей Иванович,филологический наука вопрос теория практика вы...,NaN,0,0,1,...,0,0,0,0,0,0,0,0,3449,IT
2307,2023,СОЦИАЛЬНЫЕ СЕТИ: ПРАВОВОЕ РЕГУЛИРОВАНИЕ,/article/n/sotsialnye-seti-pravovoe-regulirovanie,В статье рассматривается основные понятия соци...,"Нуркасым Кылычбекович Аркабаев, Эрмек Мамытови...",ош мамлекеттик университетинин жарчысый вестни...,УДК: 004.738.5,0,0,0,...,0,0,0,0,0,0,0,0,3790,IT
2308,2025,тест,0,тест,"Тест А.Б., Тест В.Г.",тест,no,0,0,0,...,0,0,0,0,0,0,0,0,1,IT


Функция для поиска всех соавторов:

In [35]:
def get_coauthors(df, author_name):
    coauthors = set()
    for authors_list in df['authors'].str.split(', '):
        if author_name in authors_list:
            coauthors.update(authors_list)
    coauthors.discard(author_name)  # удаляем самого автора
    return list(coauthors)

get_coauthors(df, 'Тест А.Б.')

['Тест Е.Д.', 'Тест В.Г.']

Векторизация текста:

In [37]:
def vectorize_texts(texts):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts)
    return tfidf_matrix, vectorizer

Схожесть между целевыми статьями и всеми остальными

In [40]:
def calculate_similarity(tfidf_matrix, target_indices, all_indices):
    similarities = cosine_similarity(tfidf_matrix[target_indices], tfidf_matrix[all_indices])
    return similarities.max(axis=0)

Основная функция с рекомендацией

In [43]:
def recommend_new_coauthors(df, author_name):
    
    target_mask = df['authors'].str.contains(author_name, regex=False) #данные автора
    target_indices = df[target_mask].index.tolist()
    
    if not target_indices:
        return {"error": "Автор не найден"}
    
    current_coauthors = get_coauthors(df, author_name) #текущие соавторы
    
    tfidf_matrix, _ = vectorize_texts(df['text']) #векторизация
    
    all_indices = df.index.tolist() 
    similarities = calculate_similarity(tfidf_matrix, target_indices, all_indices) #схожести

    results = pd.DataFrame({
        'index': all_indices,
        'similarity': similarities,
        'authors': df['authors'],
        'title': df['title']
    })
    
    results = results[
        (results['similarity'] < 1.0) &  #исключаем статьи автора
        (~results['authors'].isin([author_name]))  #исключаем соавторов
    ]
    
    #группировка по коэф схожести
    grouped = results.groupby('similarity').agg({
        'authors': lambda x: ', '.join(sorted(set(', '.join(x).split(', ')))),
        'title': lambda x: ', '.join(sorted(set(x)))
    }).reset_index().sort_values('similarity', ascending=False)
    
    recommendations = []
    used_authors = set(current_coauthors + [author_name]) #формируем рекомендации
    
    for _, row in grouped.iterrows():
        candidates = [a.strip() for a in row['authors'].split(',') if a.strip() not in used_authors]
        if candidates:
            unique_candidates = []
            for candidate in candidates:
                if candidate not in used_authors:
                    unique_candidates.append(candidate)
                    used_authors.add(candidate)
            
            if unique_candidates:
                recommendations.append({
                    'authors': ', '.join(unique_candidates),
                    'titles': row['title'],
                    'similarity': round(row['similarity'], 2)
                })
        
        if len(recommendations) >= 3:
            break
    
    #итоговый вывод
    output = {
        "Автор": author_name,
        "Текущие соавторы": current_coauthors,
        "Статьи": df[target_mask]['title'].tolist(),
        "Рекомендуемые соавторы": recommendations[:3]
    }
    
    return output

In [45]:
rec_list = recommend_new_coauthors(df, 'Большаков А. О.')
pprint.pprint(rec_list)

{'Автор': 'Большаков А. О.',
 'Рекомендуемые соавторы': [{'authors': 'Воеводин Вл В., Жолудев Ю. А., '
                                        'Соболев С. И., Стефанов К. С.',
                             'similarity': 0.44,
                             'titles': 'Эволюция системы метакомпьютинга '
                                       'X-Com'},
                            {'authors': 'Здитовец А. Л.',
                             'similarity': 0.43,
                             'titles': 'ОСНОВНЫЕ ТЕХНОЛОГИИ И ФРЕЙМВОРКИ ДЛЯ '
                                       'БЕКЕНД-РАЗРАБОТКИ НА JAVA'},
                            {'authors': 'Аньчков Максим Геннадьевич, Иванов '
                                        'Сергей Владимирович',
                             'similarity': 0.4,
                             'titles': 'Разработка и реализация алгоритмов '
                                       'передачи данных в телекоммуникационных '
                                       'системах'

In [360]:
#супер красивый вывод от нейросети, оставлю для доработки
def print_recommendations(result):
    print("=" * 70)
    print(f"РЕКОМЕНДАЦИИ ДЛЯ АВТОРА: {result['author']}".center(70))
    print("=" * 70)
    
    print("\n📚 Статьи автора:")
    for i, article in enumerate(result['articles'], 1):
        print(f"   {i}. {article}")
    
    print("\n🤝 Текущие соавторы:")
    print("\n".join(f"   - {coauth}" for coauth in result['current_coauthors']) or "   Нет соавторов")
    
    print("\n🔥 Рекомендуемые новые соавторы:")
    for i, rec in enumerate(result['recommendations'], 1):
  #      print(rec)
        authors = "".join(rec['author'])
        articles = "\n      ".join(rec['articles'])
        print(f"\n{i}. {authors}")
        print(f"   📑 Статьи: {articles}")
        print(f"   🎯 Схожесть: {rec['score']:.4f}")
    print("=" * 70)

In [362]:
result = get_author_recommendations(df, "Большаков А. О.")
print_recommendations(result)

               РЕКОМЕНДАЦИИ ДЛЯ АВТОРА: Большаков А. О.               

📚 Статьи автора:
   1. АВТОМАТИЗАЦИЯ ПРОЦЕССОВ МОНИТОРИНГА И ИНВЕНТАРИЗАЦИИ ИНФОРМАЦИОННО-ТЕХНОЛОГИЧЕСКОЙ ИНФРАСТРУКТУРЫ, ПРИМЕНЯЕМОЙ В УЧЕБНОМ ПРОЦЕССЕ

🤝 Текущие соавторы:
   - Леонтьев А. С.

🔥 Рекомендуемые новые соавторы:

1. Воеводин Вл В.
   📑 Статьи: Эволюция системы метакомпьютинга X-Com
   🎯 Схожесть: 0.4426

2. Здитовец А. Л.
   📑 Статьи: ОСНОВНЫЕ ТЕХНОЛОГИИ И ФРЕЙМВОРКИ ДЛЯ БЕКЕНД-РАЗРАБОТКИ НА JAVA
   🎯 Схожесть: 0.4311

3. Иванов Сергей Владимирович
   📑 Статьи: Разработка и реализация алгоритмов передачи данных в телекоммуникационных системах
   🎯 Схожесть: 0.4032
